In [9]:
import pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [10]:
# Names of finals cols
names = ["Nation", "Population", "Total military deaths from all causes", "Civilian deaths due to direct military action", 
"Civilian deaths due to indirect military action","Total deaths", 'Deaths as percent of population', 'Military wounded']

#################################################
# Cleaning WW1 Data
#################################################

# get the response in the form of html
ww1_url= "https://en.wikipedia.org/wiki/World_War_I_casualties"
ww1_df = pd.read_html(ww1_url)[0]
ww1_df.columns = ww1_df.columns.droplevel(1)  #remove multi-index layer

#Drop non-duplicated data
ww1_df = ww1_df.drop(['Combat deaths and missing in action (included in total military deaths)'], axis=1) 

#Drop unused rows
ww1_df = ww1_df[ww1_df['Nation'] != 'Central Powers']
ww1_df = ww1_df[ww1_df['Nation'] != 'TotalAllied Powers']
ww1_df = ww1_df[ww1_df['Nation'] != 'TotalCentral Powers']
ww1_df = ww1_df[ww1_df['Nation'] != 'Neutral nations']
ww1_df = ww1_df[ww1_df['Nation'] != 'Grand total']
# ww1_df = ww1_df[ww1_df['Nation'] != 'Approx. totals']

# Set Columns names
ww1_df.columns = names

# Removes all things in brackets and converts to standard numbers
ww1_df['Population'] = ww1_df['Population'].str.replace('(\[.*\])', "").astype(float) * 1_000_000  

# Removes things in brackets and parenthesis
ww1_df['Total military deaths from all causes'] = ww1_df['Total military deaths from all causes'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
# Averages the values
ww1_df['Total military deaths from all causes'] = ww1_df['Total military deaths from all causes'].str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# Removes things in brackets and parenthesis
ww1_df['Civilian deaths due to indirect military action'] = ww1_df['Civilian deaths due to indirect military action'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
# Averages the values
ww1_df['Civilian deaths due to indirect military action'] = ww1_df['Civilian deaths due to indirect military action'].str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()


# Removes things in brackets and parenthesis
ww1_df['Civilian deaths due to direct military action'] = ww1_df['Civilian deaths due to direct military action'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
# Averages the values
ww1_df['Civilian deaths due to direct military action'] = ww1_df['Civilian deaths due to direct military action'].str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# Removes things in brackets and parenthesis
ww1_df['Total deaths'] = ww1_df['Total deaths'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
# Averages the values
ww1_df['Total deaths'] = ww1_df['Total deaths'].str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# Removes things in brackets and parenthesis
ww1_df["Military wounded"] = ww1_df["Military wounded"].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
#directly fix a miscode
ww1_df["Military wounded"][6] = '1,675,000'
# Standardized NaNs
ww1_df["Military wounded"] = ww1_df["Military wounded"].str.replace('—', "NaN")
# Averages the values
ww1_df["Military wounded"] = ww1_df["Military wounded"].str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# clean Percentages
ww1_df["Deaths as percent of population"] = ww1_df["Deaths as percent of population"].replace('See footnote', 'NaN').str.replace('%', '').str.split('to', expand=True).astype(float).mean(axis=1) / 100

# clean nation names
ww1_df['Nation'] = ww1_df['Nation'].str.replace("( .)$", "").str.replace("( ..)$", "")
# directly fix an entry
ww1_df['Nation'][6] = 'United Kingdom'
ww1_df['Nation'][7] = 'British Empire'

# Add war index col
ww1_df["War"] = "World War 1"

In [150]:
#################################################
# Cleaning WW2 Data
#################################################

# get the response in the form of html
ww2_url= "https://en.wikipedia.org/wiki/World_War_II_casualties"
ww2_df = pd.read_html(ww2_url)[0]

#Drop non-duplicated data
ww2_df = ww2_df.drop(columns = ww2_df.columns[7])

#Set column names
ww2_df.columns = names

# drop unused rows
ww2_df = ww2_df[ww2_df['Nation'] != 'Austria (Unified with Germany)C']
ww2_df = ww2_df[ww2_df['Nation'] != 'Other nationsBH']
ww2_df = ww2_df[ww2_df['Nation'] != 'SpainAZ']
# ww2_df = ww2_df[ww2_df['Nation'] != 'Approx. totals']
ww2_df = ww2_df[ww2_df['Nation'] != 'South Seas Mandate (Japanese Colony)AX']
ww2_df = ww2_df[ww2_df['Nation'] != 'IrelandZ']
ww2_df = ww2_df[ww2_df['Nation'] != 'NepalAL']
ww2_df = ww2_df[ww2_df['Nation'] != 'South Seas Mandate (Japanese Colony)AX']
ww2_df = ww2_df[ww2_df['Nation'] != 'South Seas Mandate (Japanese Colony)AX']

# Clean 'Nation'
temp = ww2_df['Nation'].str.replace('(\(.*\))', "").str.strip().str.replace('[A-Z]{1,2}$', '').str.strip()
temp[58] = 'United Kingdom'
ww2_df['Nation'] = temp

# Clean 'Population'
ww2_df['Population'] = ww2_df['Population'].str.replace('(\[.{0,3}\])', "").str.replace(',', "").astype(float)
ww2_df['Population']= ww2_df['Population'].apply(lambda x: '%.0f' % x)  # Suppress scientific notation

# clean 'Total military deaths from all causes'
temp = ww2_df['Total military deaths from all causes'].str.replace('(\[.{0,3}\])', "").str.replace('(\(.*\))', "").str.replace('\+', "")
temp[28] = '319,200 to 341,000'
temp[30] = 'NaN'
temp[33] = '2,905'
temp[35] = 'NaN'
temp[59] = '407,300'
ww2_df['Total military deaths from all causes'] = temp.str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# clean 'Civilian deaths due to direct military action'
temp = ww2_df['Civilian deaths due to direct military action'].str.replace('(\[.{0,3}\])', "").str.replace('(\(.*\))', "")
temp[59] = '12,100'  # remove footnote artifact
temp[19] = '1,500,000to 3,000,000' # remove footnote artifact
ww2_df['Civilian deaths due to direct military action'] = temp.str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# clean 'Civilian deaths due to indirect military action'
temp = ww2_df['Civilian deaths due to indirect military action'].str.replace('(\[.{0,3}\])', "").str.replace('(\(.*\))', "")
temp = temp.str.replace('and','to') 
ww2_df['Civilian deaths due to indirect military action'] = temp.str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# clean 'Total deaths'
temp = ww2_df['Total deaths'].str.replace('(\[.{0,3}\])', "").str.replace('(\(.*\))', "")
temp = temp.str.replace('–','to') 
ww2_df['Total deaths'] = temp.str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# clean 'Deaths as percent of population'
temp = ww2_df["Deaths as percent of population"]
temp[19], temp[52]= 'NaN', 'NaN'
ww2_df["Deaths as percent of population"] = temp.str.replace('–', 'to').str.split('to', expand=True).astype(float).mean(axis=1) / 100

# clean 'Military wounded'
temp = ww2_df['Military wounded'].str.replace('(\[.{0,3}\])', "").str.replace('(\(.*\))', "")
temp[41] = "NaN" # Recode a missing value
ww2_df['Military wounded'] = temp.str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()


# Set war index
ww2_df["War"] = "World War 2"


In [151]:
#Concat the Data
data = pd.concat([ww1_df, ww2_df])

# data.to_csv("Uncleaned_data.csv")



In [152]:
data

,Nation,Population,Total military deaths from all causes,Civilian deaths due to direct military action,Civilian deaths due to indirect military action,Total deaths,Deaths as percent of population,Military wounded,War
0,Australia,5000000.0,60740.0,NaN,NaN,60740.0,0.0120,152171.0,World War 1
1,Canada,7200000.0,60818.0,1963.0,NaN,62818.0,0.0085,149732.0,World War 1
2,India,315100000.0,69177.0,NaN,NaN,69177.0,0.0000,69214.0,World War 1
3,New Zealand,1100000.0,17386.0,NaN,NaN,17386.0,0.0155,41317.0,World War 1
4,Newfoundland,200000.0,1387.0,NaN,NaN,1387.0,0.0070,2314.0,World War 1
...,...,...,...,...,...,...,...,...,...
57,Turkey,17370000,200.0,NaN,NaN,200.0,0.0000,NaN,World War 2
58,United Kingdom,47760000,383700.0,67200.0,NaN,450900.0,0.0094,376239.0,World War 2
59,United States,131028000,407300.0,12100.0,NaN,419400.0,0.0032,671801.0,World War 2
60,Yugoslavia,15490000,373000.0,990500.0,NaN,1363500.0,0.0880,425000.0,World War 2
